In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import shap
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [58]:
def load_data(tissue_names, metadata_path, tpm_dir, transcript2gene_path, tmm_path):
    metadata = pd.read_csv(metadata_path)
    metadata['tissue'] = metadata['source name'].str.split('_').str[0]
    metadata['age'] = metadata['characteristics: age'].str.extract(r'(\d+)')[0].astype(int)
    
    # Загрузка TPM инвариантных генов для каждой ткани
    tpm_data = {}
    for tissue in tissue_names:
        tpm_file = os.path.join(tpm_dir, f"tpm_invariant_genes_{tissue}.csv")
        tpm_data[tissue] = pd.read_csv(tpm_file)
    
    # Загрузка соответствия транскриптов и генов
    transcript2gene = pd.read_csv(transcript2gene_path, sep='\t')
    
    # Загрузка TMM нормализованных данных
    tmm_data = pd.read_csv(tmm_path)
    
    return metadata, tpm_data, transcript2gene, tmm_data

In [59]:
def prepare_isoform_data(tissue, metadata, tpm_data, transcript2gene, tmm_data):
    # Получаем инвариантные гены для ткани
    invariant_genes = tpm_data[tissue]['feature'].unique()
    
    # Фильтруем транскрипты для инвариантных генов
    transcript_subset = transcript2gene[transcript2gene['gene_id'].str.split('.').str[0].isin(invariant_genes)]
    
    # Фильтруем TMM данные по этим транскриптам
    tmm_filtered = tmm_data[tmm_data['feature'].isin(transcript_subset['transcript_id'])]

    tmm_filtered = tmm_filtered.rename(columns={'feature': 'transcript_id'})
    
    # Добавляем информацию о гене
    tmm_filtered = tmm_filtered.merge(transcript_subset, on='transcript_id')
    
    # Транспонируем TMM данные
    tmm_long = tmm_filtered.melt(id_vars=['transcript_id', 'transcript_name', 'gene_id', 'gene_name'], 
                                var_name='sample', value_name='tmm')
    
    # Добавляем метаданные
    tmm_long = tmm_long.merge(metadata, left_on='sample', right_on='Sample name')
    
    # Фильтруем только нужную ткань
    tmm_long = tmm_long[tmm_long['tissue'] == tissue]
    
    return tmm_long

In [60]:
def analyze_isoform_ratio(tmm_long, first_age, second_age, min_samples=5, change_threshold=10):
    # Фильтруем только нужные возраста
    tmm_long = tmm_long[tmm_long['age'].isin([first_age, second_age])].copy()
    
    # Результаты
    summary_results = []
    isoform_details = []
    significant_genes = set()
    
    # Группируем по генам
    for gene, gene_group in tmm_long.groupby('gene_name'):
        isoforms = gene_group['transcript_name'].unique()
        num_isoforms = len(isoforms)
        
        # Пропускаем гены с одной изоформой
        if num_isoforms == 1:
            continue
            
        if len(gene_group['sample'].unique()) < min_samples:
            continue
        
        # Вычисляем доли изоформ
        gene_group['total_expr'] = gene_group.groupby(['sample', 'age'])['tmm'].transform('sum')
        gene_group['isoform_percent'] = (gene_group['tmm'] / gene_group['total_expr']) * 100
        gene_group['isoform_percent'] = gene_group['isoform_percent'].fillna(0)
        
        # Разделяем данные по возрастам
        age1_data = gene_group[gene_group['age'] == first_age]
        age2_data = gene_group[gene_group['age'] == second_age]
        
        # Средние доли изоформ
        age1_isoforms = age1_data.groupby('transcript_name')['isoform_percent'].mean()
        age2_isoforms = age2_data.groupby('transcript_name')['isoform_percent'].mean()
        
        # Доминирующие изоформы
        dominant_age1 = age1_isoforms.idxmax() if not age1_isoforms.empty else None
        dominant_age2 = age2_isoforms.idxmax() if not age2_isoforms.empty else None
        
        # Определяем значимые изменения
        dominant_changed = (dominant_age1 != dominant_age2) if (dominant_age1 and dominant_age2) else False
        significant_changes = False
        
        # Проверяем изменения для каждой изоформы
        isoform_changes = []
        for isoform in set(age1_isoforms.index).union(set(age2_isoforms.index)):
            change = age2_isoforms.get(isoform, 0) - age1_isoforms.get(isoform, 0)
            isoform_changes.append({
                'isoform': isoform,
                'change': change,
                'is_significant': abs(change) >= change_threshold
            })
            if abs(change) >= change_threshold:
                significant_changes = True
        
        # Ген значим, если сменилась доминирующая изоформа и есть значительные изменения
        is_significant = dominant_changed and significant_changes
        if is_significant:
            significant_genes.add(gene)
        
        # Добавляем в summary таблицу
        summary_results.append({
            'gene': gene,
            'num_isoforms': num_isoforms,
            'dominant_isoform_age1': dominant_age1,
            'dominant_isoform_age2': dominant_age2,
            'dominant_changed': dominant_changed,
            'significant_changes': significant_changes,
            'is_significant': is_significant,
            'mean_percent_age1': round(age1_isoforms.max(), 2) if not age1_isoforms.empty else None,
            'mean_percent_age2': round(age2_isoforms.max(), 2) if not age2_isoforms.empty else None
        })
        
        # Детали по изоформам
        for isoform in set(age1_isoforms.index).union(set(age2_isoforms.index)):
            change = age2_isoforms.get(isoform, 0) - age1_isoforms.get(isoform, 0)
            isoform_details.append({
                'gene': gene,
                'isoform': isoform,
                'percent_age1': round(age1_isoforms.get(isoform, 0), 2),
                'percent_age2': round(age2_isoforms.get(isoform, 0), 2),
                'percent_change': round(change, 2),
                'change_abs': round(abs(change), 2),
                'is_significant_change': abs(change) >= change_threshold,
                'was_dominant': isoform == dominant_age1,
                'became_dominant': isoform == dominant_age2,
                'dominance_lost': (isoform == dominant_age1) and (isoform != dominant_age2),
                'dominance_gained': (isoform != dominant_age1) and (isoform == dominant_age2),
                'gene_significant': is_significant
            })
    
    # Конвертируем в DataFrame
    summary_df = pd.DataFrame(summary_results)
    isoform_df = pd.DataFrame(isoform_details)
    
    return summary_df, isoform_df, list(significant_genes)

In [61]:
def main_analysis(tissue_names, metadata_path, tpm_dir, transcript2gene_path, tmm_path, output_dir, 
                 first_age=3, second_age=21, change_threshold=10):
    
    try:
        # 1. Загрузка данных
        print("Loading data...")
        metadata, tpm_data, transcript2gene, tmm_data = load_data(
            tissue_names, metadata_path, tpm_dir, transcript2gene_path, tmm_path
        )
        
        all_results = []
        all_isoforms = []
        all_significant_genes = []
        
        # 2. Анализ для каждой ткани
        for tissue in tissue_names:
            print(f"\nAnalyzing tissue: {tissue}")
            
            try:
                # Подготовка данных
                tmm_long = prepare_isoform_data(tissue, metadata, tpm_data, transcript2gene, tmm_data)
                
                # Анализ соотношения изоформ
                summary_df, isoform_df, sig_genes = analyze_isoform_ratio(
                    tmm_long,
                    first_age=first_age,
                    second_age=second_age,
                    change_threshold=change_threshold
                )
                
                if not summary_df.empty:
                    # Добавляем информацию о ткани
                    summary_df['tissue'] = tissue
                    isoform_df['tissue'] = tissue
                    
                    # Сохраняем результаты для текущей ткани
                    tissue_dir = os.path.join(output_dir, tissue)
                    os.makedirs(tissue_dir, exist_ok=True)
                    
                    # Полные данные
                    summary_df.to_csv(os.path.join(tissue_dir, f"summary_{tissue}.csv"), index=False)
                    isoform_df.to_csv(os.path.join(tissue_dir, f"isoform_details_{tissue}.csv"), index=False)
                    
                    # Значимые результаты
                    significant_df = summary_df[summary_df['is_significant']]
                    if not significant_df.empty:
                        significant_df.to_csv(os.path.join(tissue_dir, f"significant_genes_{tissue}.csv"), index=False)
                        
                        # Топ-10 самых значимых изменений
                        top_changes = (isoform_df[isoform_df['gene_significant']]
                                       .sort_values('change_abs', ascending=False)
                                       .head(10))
                        top_changes.to_csv(os.path.join(tissue_dir, f"top_changes_{tissue}.csv"), index=False)
                        
                        print(f"Found {len(sig_genes)} significant genes in {tissue}")
                        all_significant_genes.extend([(tissue, gene) for gene in sig_genes])
                    
                    # Добавляем в общие результаты
                    all_results.append(summary_df)
                    all_isoforms.append(isoform_df)
                    
                else:
                    print(f"No valid genes found for {tissue} (all have only 1 isoform or insufficient samples)")
                    
            except Exception as e:
                print(f"Error processing tissue {tissue}: {str(e)}")
                continue
        
        # 3. Сохранение объединенных результатов
        if all_results:
            # Объединенные таблицы
            final_summary = pd.concat(all_results)
            final_isoforms = pd.concat(all_isoforms)
            
            # Сохраняем полные данные
            final_summary.to_csv(os.path.join(output_dir, "all_tissues_summary.csv"), index=False)
            final_isoforms.to_csv(os.path.join(output_dir, "all_tissues_isoforms.csv"), index=False)
            
            # Сохраняем значимые гены
            if all_significant_genes:
                sig_genes_df = pd.DataFrame(all_significant_genes, columns=['tissue', 'gene'])
                sig_genes_df.to_csv(os.path.join(output_dir, "all_significant_genes.csv"), index=False)
                
                # Сводная таблица по значимым генам
                sig_summary = (final_summary[final_summary['is_significant']]
                              .sort_values(['tissue', 'dominant_changed', 'significant_changes'], 
                                          ascending=[True, False, False]))
                sig_summary.to_csv(os.path.join(output_dir, "significant_genes_summary.csv"), index=False)
                
                print(f"\nTotal significant genes found: {len(sig_genes_df)}")
            
            print("\nAnalysis completed successfully!")
            print(f"Results saved to: {output_dir}")
            
        else:
            print("No valid results generated for any tissue")
            
    except Exception as e:
        print(f"Fatal error in main analysis: {str(e)}")
        raise

In [62]:
def get_tissue_names(directory):

    tissue_names = set()
    
    # Проверяем существование директории
    if not os.path.exists(directory):
        raise FileNotFoundError(f"Directory not found: {directory}")
    
    # Шаблон для поиска файлов
    pattern = re.compile(r'tpm_invariant_genes_(.*?)\.csv$')
    
    # Перебираем файлы в директории
    for filename in os.listdir(directory):
        match = pattern.match(filename)
        if match:
            tissue_name = match.group(1)
            tissue_names.add(tissue_name)
    
    return sorted(tissue_names)

In [63]:
tpm_dir = "filtered_genes"
TISSUE_NAMES = get_tissue_names(tpm_dir)

In [64]:
METADATA_PATH = 'metadata.csv'
TPM_DIR = 'filtered_genes'
TRANSCRIPT2GENE_PATH = 'gencode_vM22.transcript2gene.tsv'
TMM_PATH = 'tmm_filtered_salmon_trs.csv'
OUTPUT_DIR = 'isoform_analysis_results'

In [65]:
main_analysis(
        tissue_names=TISSUE_NAMES,
        metadata_path=METADATA_PATH,
        tpm_dir=TPM_DIR,
        transcript2gene_path=TRANSCRIPT2GENE_PATH,
        tmm_path=TMM_PATH,
        output_dir=OUTPUT_DIR,
        first_age=3,
        second_age=21
    )

Loading data...

Analyzing tissue: BAT
Found 84 significant genes in BAT

Analyzing tissue: Bone
Found 216 significant genes in Bone

Analyzing tissue: Brain
Found 23 significant genes in Brain

Analyzing tissue: GAT
Found 29 significant genes in GAT

Analyzing tissue: Heart
Found 40 significant genes in Heart

Analyzing tissue: Kidney
Found 36 significant genes in Kidney

Analyzing tissue: Limb_Muscle
No valid genes found for Limb_Muscle (all have only 1 isoform or insufficient samples)

Analyzing tissue: Liver
Found 62 significant genes in Liver

Analyzing tissue: Lung
Found 32 significant genes in Lung

Analyzing tissue: MAT
Found 48 significant genes in MAT

Analyzing tissue: Marrow
Found 118 significant genes in Marrow

Analyzing tissue: Pancreas

Analyzing tissue: SCAT
Found 115 significant genes in SCAT

Analyzing tissue: Skin
Found 173 significant genes in Skin

Analyzing tissue: Small_Intestine
No valid genes found for Small_Intestine (all have only 1 isoform or insufficient s

In [18]:
def load_and_map_data(summary_path, transcript2gene_path, annotation_path):
    summary = pd.read_csv(summary_path)
    transcript_map = pd.read_csv(transcript2gene_path, sep='\t')
    annotations = pd.read_csv(annotation_path, sep='\t')
    
    # 1. Связываем summary с transcript_map по transcript_name
    summary_mapped = pd.merge(
        summary,
        transcript_map[['transcript_name', 'transcript_id']],
        left_on='dominant_isoform_age1',
        right_on='transcript_name',
        how='left'
    ).rename(columns={'transcript_id': 'age1_transcript_id'})
    
    summary_mapped = pd.merge(
        summary_mapped,
        transcript_map[['transcript_name', 'transcript_id']],
        left_on='dominant_isoform_age2',
        right_on='transcript_name',
        how='left'
    ).rename(columns={'transcript_id': 'age2_transcript_id'})
    
    # 2. Собираем все уникальные transcript_id для аннотирования
    age1_ids = summary_mapped['age1_transcript_id'].dropna().unique()
    age2_ids = summary_mapped['age2_transcript_id'].dropna().unique()
    all_ids = np.union1d(age1_ids, age2_ids)
    
    # 3. Фильтруем аннотации только для нужных transcript_id
    annotations_filtered = annotations[annotations['transcript_id'].isin(all_ids)].copy()
    
    # 4. Создаём финальный датасет
    age1_data = pd.DataFrame({
        'transcript_id': age1_ids,
        'age_class': 0  # 0 для young (age1)
    })
    
    age2_data = pd.DataFrame({
        'transcript_id': age2_ids,
        'age_class': 1  # 1 для old (age2)
    })
    
    combined = pd.concat([age1_data, age2_data])
    final_data = pd.merge(combined, annotations_filtered, on='transcript_id', how='left')
    
    # Проверка качества слияния
    if final_data.empty:
        raise ValueError("После объединения данных не осталось записей. Проверьте соответствие идентификаторов.")
    
    return final_data

In [19]:
def prepare_features(data):
    """Подготавливает фичи для модели"""
    # Обработка transcript_type
    data['transcript_type'] = data['transcript_type'].apply(
        lambda x: 1 if x == 'protein_coding' else 0
    )
    
    # Выбираем нужные признаки
    features = [
        'five_prime_UTR_length',
        'three_prime_UTR_length',
        'transcript_type',
        'CDS_length',
        'exon_count',
        'total_exonic_length'
    ]
    
    # Заполняем пропуски медианами
    for col in features:
        if data[col].isnull().any():
            data[col] = data[col].fillna(data[col].median())
    
    return data[features], data['age_class'], features

In [35]:
def run_analysis():
    """Полный пайплайн анализа"""
    # 1. Загрузка и объединение данных
    print("Загрузка и объединение данных...")
    try:
        data = load_and_map_data(
            "isoform_analysis_results/significant_genes_summary.csv",
            "gencode_vM22.transcript2gene.tsv",
            "gencode_vM22.transcript_ext_data.tsv"
        )
        print(f"Успешно загружено {len(data)} изоформ")
    except Exception as e:
        print(f"Ошибка при загрузке данных: {e}")
        return
    
    # 2. Подготовка признаков
    print("Подготовка признаков...")
    X, y, feature_names = prepare_features(data)
    
    # 3. Проверка баланса классов
    print("\nРаспределение классов:")
    print(pd.Series(y).value_counts())
    

    
    # SHAP
    # 1. Разделение данных и обучение модели
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    model = RandomForestClassifier(
        n_estimators=100,
        max_depth=5,
        random_state=42,
        class_weight='balanced'
    )
    model.fit(X_train, y_train)
    
    # 2. Инициализация SHAP
    explainer = shap.TreeExplainer(model)
    
    try:
        # Попробуем новый API (для версий shap >= 0.39.0)
        shap_values = explainer(X_test)
        
        if hasattr(shap_values, 'values'):
            # Обработка объекта Explanation
            print("Обнаружен новый API SHAP (объект Explanation)")
            
            # Для бинарной классификации выбираем значения для класса 1
            if len(shap_values.shape) == 3:  # [n_samples, n_features, n_classes]
                shap_values_class1 = shap_values.values[:, :, 1]
            else:
                shap_values_class1 = shap_values.values
        else:
            # Обработка старого формата (массив numpy)
            print("Обнаружен старый API SHAP (numpy array)")
            shap_values = explainer.shap_values(X_test)
            
            if isinstance(shap_values, list):
                # Для бинарной классификации выбираем класс 1
                shap_values_class1 = shap_values[1]
            else:
                shap_values_class1 = shap_values
    except Exception as e:
        print(f"Ошибка при получении SHAP значений: {e}")
        return None, None, None

    # 3. Проверка размерностей
    print("\nПроверка размерностей:")
    print(f"SHAP values: {np.array(shap_values_class1).shape}")
    print(f"X_test: {X_test.shape}")
    
    if np.array(shap_values_class1).shape != X_test.shape:
        print("\nПредупреждение: Размерности не совпадают! Применяем корректировку...")
        try:
            # Попытка автоматической корректировки
            if len(shap_values_class1.shape) == 3:
                shap_values_class1 = shap_values_class1[:, :, 0]
            elif len(shap_values_class1.shape) == 2 and shap_values_class1.shape[0] == X_test.shape[1]:
                shap_values_class1 = shap_values_class1.T
        except:
            raise ValueError(
                f"Не удалось автоматически исправить размерности.\n"
                f"SHAP values shape: {shap_values_class1.shape}\n"
                f"X_test shape: {X_test.shape}"
            )

    # 4. Визуализация
    plt.figure(figsize=(12, 6))
    try:
        shap.summary_plot(
            shap_values_class1, 
            X_test, 
            feature_names=feature_names,
            show=False
        )
        plt.title("SHAP Values for Old Mice (21 months vs 3 months)")
        plt.tight_layout()
        plt.savefig("shap_results.png", dpi=300, bbox_inches='tight')
        plt.close()
    except Exception as e:
        print(f"Ошибка при визуализации: {e}")
        return explainer, shap_values, model

    # 5. Сохранение важности признаков
    try:
        feature_importance = pd.DataFrame({
            'feature': feature_names,
            'mean_abs_shap': np.mean(np.abs(shap_values_class1), axis=0)
        }).sort_values('mean_abs_shap', ascending=False)
        
        feature_importance.to_csv("shap_feature_importance.csv", index=False)
    except Exception as e:
        print(f"Ошибка при сохранении важности признаков: {e}")

    return explainer, shap_values, model

In [36]:
run_analysis()

Загрузка и объединение данных...
Успешно загружено 1690 изоформ
Подготовка признаков...

Распределение классов:
0    846
1    844
Name: age_class, dtype: int64
Обнаружен новый API SHAP (объект Explanation)

Проверка размерностей:
SHAP values: (338, 6)
X_test: (338, 6)


(<shap.explainers._tree.TreeExplainer at 0x7fca68b0a140>,
 .values =
 array([[[-0.00883352,  0.00883352],
         [-0.00669131,  0.00669131],
         [ 0.0004698 , -0.0004698 ],
         [-0.00305986,  0.00305986],
         [ 0.0006457 , -0.0006457 ],
         [-0.02190004,  0.02190004]],
 
        [[ 0.01020694, -0.01020694],
         [ 0.07569534, -0.07569534],
         [ 0.00071387, -0.00071387],
         [ 0.00093469, -0.00093469],
         [ 0.00707728, -0.00707728],
         [ 0.00273917, -0.00273917]],
 
        [[ 0.06191129, -0.06191129],
         [ 0.00589943, -0.00589943],
         [-0.00500759,  0.00500759],
         [-0.01995468,  0.01995468],
         [ 0.00465768, -0.00465768],
         [ 0.01704141, -0.01704141]],
 
        ...,
 
        [[-0.00970913,  0.00970913],
         [-0.00382887,  0.00382887],
         [ 0.00112754, -0.00112754],
         [ 0.00229001, -0.00229001],
         [ 0.02427157, -0.02427157],
         [-0.000687  ,  0.000687  ]],
 
        [[ 0.007